AR non-anomaly composites of different variables using different cEOF tests (e.g. DJF, MAM, DJF-MAM). 

Creates significant differences plots for IVT and upper level plots. (e.g. El Nino vs Neutral conditions for each AR type)


## Imports

In [1]:
# Standard Python modules
import os, sys
import yaml
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
# matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib import rcParams
from matplotlib.colors import ListedColormap
# plot styles/formatting
import seaborn as sns
import cmocean.cm as cmo
import cmocean
# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes

# Extra 
from scipy.ndimage import gaussian_filter    # smoothing contour lines
import metpy.calc as mpcalc
from metpy.units import units
from IPython.display import Image, display

# import personal modules
# Path to modules
sys.path.append('../modules')
# Import my modules
from plotter import draw_basemap, add_subregion_boxes, make_cmap
import nclcmaps as nclc
from timeseries import persistence, select_months, create_list_all_dates
from teleconnections import build_teleconnection_df, mjo_index
from statistical_tests import xr_zscore_diff_mean
from ar_funcs import ar_daily_df

In [2]:
# Set up paths

path_to_data = '/home/nash/DATA/data/'                            # project data -- read only
path_to_out  = '/home/nash/DATA/repositories/AR_types/out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '/home/nash/DATA/repositories/AR_types/figs/'      # figures


In [3]:
# Set a default font for all matplotlib text 
rcParams['font.family'] = 'sans-serif'   # font family = 'sans-serif'
rcParams['font.sans-serif'] = 'Arial'    # default sans-serif font = 'Arial'


## Data

In [4]:
# choose season
ssn = 'djfmam'
ceofid = 'IVT'
neofs = 3 # choose number of eofs
nk = 3 # choose number of clusters

out_path = path_to_out + ceofid + '/' + ssn + '/' + 'neof' + str(neofs) + '/k' + str(nk) + '/'
fig_path = path_to_figs + ceofid + '/' + ssn + '/'+ 'neof' + str(neofs) + '/k' + str(nk) + '/'

In [5]:
# import configuration file for plot dictionary
yaml_doc = '../data/plt_config.yml'

config = yaml.load(open(yaml_doc), Loader=yaml.SafeLoader)
#select dictionaries - choose var, anom/nanom, and season
# upper_ precip_ ivt_ and non_anom anom
plot_dict_upper = config['upper_diff']
plot_dict_ivt = config['ivt_diff']
plot_dict_prec = config['precip_diff']
plot_dict_iwv = config['iwv_diff']
plot_dict_500 = config['lower_diff']

plot_dicts = [plot_dict_upper, plot_dict_500, plot_dict_ivt, plot_dict_prec]

# djf_dict mam_dict djfmam_dict
ar_dict = config[ssn]

### AR days and Climate Indices

In [6]:
outpath = path_to_out + ceofid + '/' + ssn + '/' + 'neof' + str(neofs) + '/k' + str(nk) + '/'
start_date = '1979-12-01'
end_date = '2019-05-31'

# climate index df
tele = build_teleconnection_df('daily', 'COND', start_date, end_date)
mjo = mjo_index('daily', ar_dict['start_date'], ar_dict['end_date'])
tele['MJO'] = mjo['COND']
df_index = select_months(tele, 12, 5)

df = ar_daily_df(ssn, nk, outpath)
# combine ar df with tele df
# join indices with AR count
new_df = df.join(df_index)
new_df

,AR_CAT1,AR_CAT2,AR_CAT3,AR_ALL,AR_CAT,AO,PDO,ENSO,SH,MJO
date,,,,,,,,,,
1979-12-01 09:00:00,0.0,1.0,0.0,1.0,2,1,0,0,0,1
1979-12-02 09:00:00,0.0,0.0,1.0,1.0,3,1,0,0,0,1
1979-12-03 09:00:00,0.0,0.0,0.0,0.0,0,1,0,0,0,1
1979-12-04 09:00:00,0.0,0.0,0.0,0.0,0,1,0,0,0,1
1979-12-05 09:00:00,0.0,0.0,0.0,0.0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
2019-05-27 09:00:00,0.0,0.0,0.0,0.0,0,0,0,1,1,1
2019-05-28 09:00:00,0.0,0.0,0.0,0.0,0,-1,0,1,1,1
2019-05-29 09:00:00,0.0,0.0,0.0,0.0,0,-1,0,1,1,1


In [7]:
## correlation matrix
new_df.corr()

,AR_CAT1,AR_CAT2,AR_CAT3,AR_ALL,AR_CAT,AO,PDO,ENSO,SH,MJO
AR_CAT1,1.000000,-0.107700,-0.109186,0.527482,0.143953,0.009797,0.014764,0.004318,-0.015905,-0.017522
AR_CAT2,-0.107700,1.000000,-0.103618,0.500582,0.456221,0.056972,0.027013,0.057025,-0.030833,0.001328
AR_CAT3,-0.109186,-0.103618,1.000000,0.507490,0.786536,0.041188,-0.011516,0.012533,-0.025868,0.013858
AR_ALL,0.527482,0.500582,0.507490,1.000000,0.896265,0.069593,0.019673,0.047445,-0.047048,-0.001877
AR_CAT,0.143953,0.456221,0.786536,0.896265,1.000000,0.072235,0.010001,0.045401,-0.045447,0.007661
AO,0.009797,0.056972,0.041188,0.069593,0.072235,1.000000,-0.169020,-0.025015,-0.101567,0.005215
PDO,0.014764,0.027013,-0.011516,0.019673,0.010001,-0.169020,1.000000,0.397739,0.029224,-0.076604
ENSO,0.004318,0.057025,0.012533,0.047445,0.045401,-0.025015,0.397739,1.000000,-0.224652,-0.020668
SH,-0.015905,-0.030833,-0.025868,-0.047048,-0.045447,-0.101567,0.029224,-0.224652,1.000000,0.040454
MJO,-0.017522,0.001328,0.013858,-0.001877,0.007661,0.005215,-0.076604,-0.020668,0.040454,1.000000


In [8]:
from scipy.stats import pearsonr
import pandas as pd

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

calculate_pvalues(new_df)

,AR_CAT1,AR_CAT2,AR_CAT3,AR_ALL,AR_CAT,AO,PDO,ENSO,SH,MJO
AR_CAT1,0.0,0.0,0.0,0.0,0.0,0.403,0.2075,0.7124,0.1745,0.1347
AR_CAT2,0.0,0.0,0.0,0.0,0.0,0.0,0.0211,0.0,0.0085,0.9098
AR_CAT3,0.0,0.0,0.0,0.0,0.0,0.0004,0.3255,0.2847,0.0272,0.2368
AR_ALL,0.0,0.0,0.0,0.0,0.0,0.0,0.093,0.0001,0.0001,0.8727
AR_CAT,0.0,0.0,0.0,0.0,0.0,0.0,0.3932,0.0001,0.0001,0.5131
AO,0.403,0.0,0.0004,0.0,0.0,0.0,0.0,0.0327,0.0,0.6562
PDO,0.2075,0.0211,0.3255,0.093,0.3932,0.0,0.0,0.0,0.0126,0.0
ENSO,0.7124,0.0,0.2847,0.0001,0.0001,0.0327,0.0,0.0,0.0,0.0776
SH,0.1745,0.0085,0.0272,0.0001,0.0001,0.0,0.0126,0.0,0.0,0.0006
MJO,0.1347,0.9098,0.2368,0.8727,0.5131,0.6562,0.0,0.0776,0.0006,0.0


### ERA5 renalysis

In [8]:
rename_dict_upper = {'z': 'H', 
                     'u': 'U250', 
                     'v':'V250', 
                     'q': 'QV',
                     'latitude': 'lat',
                     'longitude': 'lon'}

rename_dict_prec = {'mtpr': 'prec', 
                    'latitude': 'lat',
                    'longitude': 'lon'}

rename_dict_iwv = {'latitude': 'lat',
                    'longitude': 'lon'}

rename_dict_lower = {'z': 'H', 
                     'u': 'U500', 
                     'v':'V500', 
                     'q': 'QV',
                     'latitude': 'lat',
                     'longitude': 'lon'}

rename_dict_ivt = {'p71.162': 'ivte', 
                   'p72.162': 'ivtn', 
                   'latitude': 'lat',
                   'longitude': 'lon'}

In [9]:
def preprocess(ds):
    '''keep only selected lats and lons'''
    if plot_dict['name'] == 'huv250':
        subset = ds.sel(latitude=slice(latmax, latmin), longitude=slice(lonmin, lonmax), level=plot_dict['lev'])
        subset = subset.rename(rename_dict_upper)
        subset = subset.drop(['QV'])
    if plot_dict['name'] == 'prec':
        subset = ds.sel(latitude=slice(latmax, latmin), longitude=slice(lonmin, lonmax))
        subset = subset.rename(rename_dict_prec)
    if plot_dict['name'] == 'huv500':
        subset = ds.sel(latitude=slice(latmax, latmin), longitude=slice(lonmin, lonmax), level=plot_dict['lev'])
        subset = subset.rename(rename_dict_lower)
        subset = subset.drop(['QV', 'H'])
    if plot_dict['name'] == 'iwv':
        subset = ds.sel(latitude=slice(latmax, latmin), longitude=slice(lonmin, lonmax))
        subset = subset.rename(rename_dict_iwv)
        subset = subset.drop(['tcrw', 'tcsw', 'tcw'])
    if plot_dict['name'] == 'ivt':
        subset = ds.sel(latitude=slice(latmax, latmin), longitude=slice(lonmin, lonmax))
        subset = subset.rename(rename_dict_ivt)
        
    return subset

f = []

# Select lat/lon grid bbox [latmin, latmax, lonmin, lonmax]
lonmin = 20
lonmax = 120
latmin = 5
latmax = 55

    
for p, plot_dict in enumerate(plot_dicts):

    # # open data  
    f.append(xr.open_mfdataset(path_to_data + plot_dict['fname'] , preprocess=preprocess, combine='by_coords'))

ds = xr.merge(f[:3], compat='override')
print('ds size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))
# f[0]['H'] = f[0]['H']/(9.80665), # convert to geopotential height (m)
# f[1]['prec'] = f[1]['prec']*86400 # convert to mm day-1



KeyboardInterrupt: 

In [ ]:
# import precipitation averages for AR Types
fname = path_to_out+'precip_djfmam_h500_eof2_k3.nc'
precipitation = xr.open_dataset(fname)
precipitation

### Combine with data

In [ ]:
%%time
df = new_df
ds_lst = [ds, f[3]]
for i, ds in enumerate(ds_lst):
    # Trim date range
    idx = slice(ar_dict['start_date'], ar_dict['end_date'])
    ds = ds.sel(time=idx)
    
    # Select months
    if ar_dict['mon_s'] > ar_dict['mon_e']:
        idx = (ds.time.dt.month >= ar_dict['mon_s']) | (ds.time.dt.month <= ar_dict['mon_e'])
    else:
        idx = (ds.time.dt.month >= ar_dict['mon_s']) & (ds.time.dt.month <= ar_dict['mon_e'])
    ds = ds.sel(time=idx)
    
    # Combine AR Cat data w/ reanalysis data
    # Add ar time series to the ERA dataset
    cols = ['AR_CAT', 'AO', 'PDO', 'ENSO', 'SH', 'MJO']
    
    for k, col in enumerate(cols):
        ds[col] = ('time', df[col])

    ds = ds.set_coords(tuple(cols))
    
    # select only AR days
    idx = (ds.AR_CAT > 0)
    ds = ds.sel(time=idx)
    print('ds size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))
    ds = ds.load()
    ds_lst[i] = ds

In [ ]:
## calculate 500 hPa wind speed and IVT
ds = ds_lst[0]
ds = ds.assign(wspd=lambda ds: np.sqrt(ds.U500**2 + ds.V500**2))
ds = ds.assign(ivt=lambda ds: np.sqrt(ds.ivte**2 + ds.ivtn**2))
ds_lst[0] = ds
ds_lst[0]

### Compute AR Composites

In [ ]:
%%time
# Compare condition (e.g. Type 1 ARs during El Nino) to DJFMAM anomalies
artype_lst = [3, 3, 3]
cli_lst = ['ENSO', 'AO', 'SH']
cond_lst1 = [1]*3 + [-1]*3
cond1= 1
cond2 = -1

com_diff = []
com_pval = []
mjo_diff = []
mjo_pval = []


for m, ds_nm in enumerate(ds_lst):
    ds = ds_nm
    for k, (cli, ar_type) in enumerate(zip(cli_lst,  artype_lst)):

        idx = (ds[cli] == cond1) & (ds['AR_CAT'] == ar_type)
        data1 = ds.sel(time=idx)
        idx = (ds[cli] == cond2) & (ds['AR_CAT'] == ar_type)
        data2 = ds.sel(time=idx)

        diff, pval = xr_zscore_diff_mean(data1, data2)

        com_diff.append(diff)
        com_pval.append(pval)
        
        # MJO
        idx = (ds['MJO'] == cond1) & (ds['AR_CAT'] == 1)
        data1 = ds.sel(time=idx)
        idx = (ds['MJO'] == 0) & (ds['AR_CAT'] == 1)
        data2 = ds.sel(time=idx)

        diff2, pval2 = xr_zscore_diff_mean(data1, data2)

        mjo_diff.append(diff2)
        mjo_pval.append(pval2)

### Figures

In [ ]:
plot_names = ['TYPE3']
a=3
b=6
c=9

plot_ds = [com_diff[:a]]
pval_ds = [com_pval[:a]]

plot_prec = [com_diff[a:]]
pval_prec = [com_pval[a:]]

### Common Plot Properties

In [ ]:
# Set seaborn plot style
sns.set()
sns.set_style("ticks", {'patch.force_edgecolor':False})

# Set projections
datacrs = ccrs.PlateCarree()   # data/source
mapcrs = ccrs.PlateCarree()    # map/destination

# Set tick/grid locations
dx = np.arange(lonmin,lonmax+20,20)
dy = np.arange(latmin,latmax+20,20)

# subregion info 
# [ymin, xmin]
sr_xy = [[65, 30], [75, 25], [85, 20]]
# width of subregion
sr_width = [10, 10, 10]
# height of subregion
sr_height = [10, 10, 10]


### Composite Plots

In [ ]:
for i, cliindex in enumerate(plot_names):
    print(cliindex)
    filepath = fig_path + 'composite_diff_' + cliindex + '.png'
    nrows = nk
    ncols = 2
    sig_level = 0.05
    artype_lst = [3, 3, 3]
    row_lbl = ['El Nino - La Nina', 'AO+ AO-', 'SH+ SH-']
    col_lbl1 = ['(a)', '(b)', '(c)']
    col_lbl2 = ['(d)', '(e)', '(f)']
    col_lbl3 = ['(g)', '(h)', '(i)']
    ext1 = [20, 120, 5, 55]

    # Create figure
    fig = plt.figure(figsize=(15, 10))

    # Set up Axes Grid
    axes_class = (GeoAxes,dict(map_projection=mapcrs))
    axgr = AxesGrid(fig, [0.1, 0.1, 0.66, 0.8], axes_class=axes_class,
                nrows_ncols=(nrows, 2), axes_pad = 0.45,
                cbar_location='bottom', cbar_mode='edge',
                cbar_pad=0.25, cbar_size='7%',label_mode='',
                direction='column',
                aspect=True,
                share_all = False)

    axgr2 = AxesGrid(fig, [0.75, 0.1, 0.33, 0.8], axes_class=axes_class,
                nrows_ncols=(nrows, 1), axes_pad = 0.45,
                cbar_location='bottom', cbar_mode='edge',
                cbar_pad=0.25, cbar_size='7%',label_mode='',
                direction='column',
                aspect=True,
                share_all = False)

    #################################
    ########## UPPER PLOTS ##########
    #################################
    for k, (ax, ar_type) in enumerate(zip(axgr[0:nrows], artype_lst)):
        data = plot_ds[i][k]
        pval = pval_ds[i][k]
        plot_dict = plot_dict_upper
        # lat/lon arrays
        lats = data.lat.values
        lons = data.lon.values    
        ax = draw_basemap(ax, extent=ext1, xticks=dx, yticks=dy, left_lats=False, right_lats=True)

        uvec = data.U250.values
        vvec = data.V250.values
        hgts = data.H.values/(9.80665) # convert to geopotential height (m)

        uvec_mask = data.U250.where((pval.U250 <= sig_level) | (pval.V250 <= sig_level)).values
        vvec_mask = data.V250.where((pval.U250 <= sig_level) | (pval.V250 <= sig_level)).values
        hgts_mask = data.H.where((pval.H <= sig_level)).values/(9.80665) # convert to geopotential height (m)

       # Contour Filled (hgts significant)                           
        cflevs = clevs = np.arange(-60, 75, 15)
        cf = ax.contourf(lons, lats, hgts_mask, transform=datacrs,
                         levels=cflevs, cmap=plot_dict['cmap'], alpha=0.9, extend='both') 

        # Wind barbs / vectors
        Q = ax.quiver(lons, lats, uvec, vvec, transform=datacrs, 
                      color='k', regrid_shape=17, pivot='middle',
                      angles='xy', scale_units='xy', scale=plot_dict['quiver_scale'], units='xy')

        # quiver key
        qk = ax.quiverkey(Q, 0.7, 1.07, plot_dict['quiver_key'], plot_dict['quiver_key_lbl'], labelpos='E',
                              coordinates='axes', fontproperties={'size': 8.0})

        # Contour Lines (250-hPa Heights ALL)
        clevs = cflevs
        cs = ax.contour(lons, lats, hgts, transform=datacrs,
                        levels=clevs, colors='k', linewidths=0.8)

        # Contour Lines (Precip conditions)
        prec = precipitation.prec.sel(ar=ar_type)
        clevs = np.array([5, 10, 20])
        prec_sm = gaussian_filter(prec.values, sigma=2.0)
        cs = ax.contour(prec.lon, prec.lat, prec_sm, transform=datacrs,
                        levels=clevs, colors='k', linewidths=1.25)
        # subtitles
        ax.set_title(col_lbl1[k], loc='left',fontsize=13)
        # Row labels
        ax.text(-0.07, 0.55, row_lbl[k], va='bottom', ha='center',
            rotation='vertical', rotation_mode='anchor', fontsize=13,
            transform=ax.transAxes)

        # Colorbar (single)
        cb = fig.colorbar(cf, axgr.cbar_axes[0], orientation='horizontal', drawedges=True)
        cb.set_label(plot_dict['cb_label'])

    ################################
    ########## IVT PLOTS ###########
    ################################
    for k, (ax, ar_type) in enumerate(zip(axgr[nrows:nrows+nrows], artype_lst)):
        data = plot_ds[i][k]
        pval = pval_ds[i][k]
        plot_dict = plot_dict_ivt
        # lat/lon arrays
        lats = data.lat.values
        lons = data.lon.values    
        ax = draw_basemap(ax, extent=ext1, xticks=dx, yticks=dy, left_lats=False, right_lats=True)

        uvec = data.ivte.values
        vvec = data.ivtn.values
        ivt = data.ivt.values
        ivt_mask = data.ivt.where((pval.ivt <= sig_level)).values
    

       # Contour Filled (significant)                           
        cflevs =  np.arange(plot_dict['cflevs'][0], plot_dict['cflevs'][1], plot_dict['cflevs'][2])
        cf = ax.contourf(lons, lats, ivt_mask, transform=datacrs,
                         levels=cflevs, cmap=plot_dict['cmap'], alpha=0.9, extend='both') 

        # Wind barbs / vectors
        Q = ax.quiver(lons, lats, uvec, vvec, transform=datacrs, 
                      color='k', regrid_shape=17, pivot='middle',
                      angles='xy', scale_units='xy', scale=plot_dict['quiver_scale'], units='xy')

        # quiver key
        qk = ax.quiverkey(Q, 0.7, 1.07, plot_dict['quiver_key'], plot_dict['quiver_key_lbl'], labelpos='E',
                              coordinates='axes', fontproperties={'size': 8.0})

        # Contour Lines (Precip conditions)
        prec = precipitation.prec.sel(ar=ar_type)
        clevs = np.array([5, 10, 20])
        prec_sm = gaussian_filter(prec.values, sigma=2.0)
        cs = ax.contour(prec.lon, prec.lat, prec_sm, transform=datacrs,
                        levels=clevs, colors='k', linewidths=1.25)
        # subtitles
        ax.set_title(col_lbl2[k], loc='left',fontsize=13)

        # Colorbar (single)
        cb = fig.colorbar(cf, axgr.cbar_axes[1], orientation='horizontal', drawedges=True)
        cb.set_label(plot_dict['cb_label'])
    
    ################################
    ########## WIND PLOTS ###########
    ################################
    for k, (ax, ar_type) in enumerate(zip(axgr2, artype_lst)):
        data = plot_ds[i][k]
        pval = pval_ds[i][k]
        plot_dict = plot_dict_500
        # lat/lon arrays
        lats = data.lat.values
        lons = data.lon.values    
        ax = draw_basemap(ax, extent=[60, 105, 20, 45], xticks=dx, yticks=dy, left_lats=False, right_lats=True)

        # 500-hPa Winds (m/s)
        uwnd = data.U500.values
        vwnd = data.V500.values

        # 500-hPa Winds (knots)
        wspd = data.wspd.where((pval.wspd <= sig_level)).values * units('m/s')
        wspd_kt = wspd.to('kt')
    

        # Contour Filled (significant)                           
        cflevs = np.arange(-8, 9, 1)
        cf = ax.contourf(lons, lats, wspd_kt, transform=datacrs,
                         levels=cflevs, cmap='PuOr', alpha=0.9, extend='both') 

        # Wind barbs / vectors
        Q = ax.quiver(lons, lats, uwnd, vwnd, transform=datacrs, 
                      color='k', regrid_shape=20, pivot='middle',
                      angles='xy', scale_units='xy', scale=1, units='xy')

        # quiver key
        qk = ax.quiverkey(Q, 0.7, 1.07, plot_dict['quiver_key'], plot_dict['quiver_key_lbl'], labelpos='E',
                              coordinates='axes', fontproperties={'size': 8.0})

        # Contour Lines (Precip conditions)
        prec = precipitation.prec.sel(ar=ar_type)
        clevs = np.array([5, 10, 20])
        prec_sm = gaussian_filter(prec.values, sigma=2.0)
        cs = ax.contour(prec.lon, prec.lat, prec_sm, transform=datacrs,
                        levels=clevs, colors='k', linewidths=1.25)
        # subtitles
        ax.set_title(col_lbl3[k], loc='left',fontsize=13)

        # Colorbar (single)
        cb = fig.colorbar(cf, axgr2.cbar_axes[0], orientation='horizontal', drawedges=True)
        cb.set_label(plot_dict['cb_label'])
        
    # Save figure
    plt.savefig(filepath, dpi=300, bbox_inches='tight')

    # Show
    plt.show()


In [ ]:
for i, cliindex in enumerate(plot_names):
    print(cliindex)
    filepath = fig_path + 'composite_diff_prec' + cliindex + '.png'
    nrows = nk
    ncols = 1
    sig_level = 0.05
    artype_lst = [3, 3, 3]
    row_lbl = ['El Nino - La Nina', 'AO+ AO-', 'SH+ SH-']
    col_lbl1 = ['(a)', '(b)', '(c)']
    ext1 = [20, 120, 5, 55]

    # Create figure
    fig = plt.figure(figsize=(15, 10))

    # Set up Axes Grid
    axes_class = (GeoAxes,dict(map_projection=mapcrs))
    axgr = AxesGrid(fig, 111, axes_class=axes_class,
                nrows_ncols=(nrows, ncols), axes_pad = 0.45,
                cbar_location='bottom', cbar_mode='edge',
                cbar_pad=0.25, cbar_size='7%',label_mode='',
                direction='column',
                aspect=True,
                share_all = False)
    #################################
    ########## PREC PLOTS ##########
    #################################
    for k, (ax, ar_type) in enumerate(zip(axgr, artype_lst)):
        data = plot_prec[i][k]
        pval = pval_prec[i][k]
        plot_dict = plot_dict_upper
        # lat/lon arrays
        lats = data.lat.values
        lons = data.lon.values    
        ax = draw_basemap(ax, extent=[60, 105, 20, 45], xticks=dx, yticks=dy, left_lats=False, right_lats=True)

        prec = data.prec.values*86400 # convert to mm day-1
        prec_mask = data.prec.where((pval.prec <= sig_level)).values*86400 # convert to mm day-1

       # Contour Filled                           
        cflevs = clevs = np.arange(-5, 6, 1)
        cf = ax.contourf(lons, lats, prec, transform=datacrs,
                         levels=cflevs, cmap=plot_dict['cmap'], alpha=0.9, extend='both') 

        # Stippling Significant values
#         cs = ax.contourf(lons, lats, prec_mask, colors='none', hatches=['.'])
        
        # Contour Lines (Precip conditions)
        prec = precipitation.prec.sel(ar=ar_type)
        clevs = np.array([5, 10, 20])
        prec_sm = gaussian_filter(prec.values, sigma=2.0)
        cs = ax.contour(prec.lon, prec.lat, prec_sm, transform=datacrs,
                        levels=clevs, colors='k', linewidths=1.25)
        # subtitles
        ax.set_title(col_lbl1[k], loc='left',fontsize=13)

        # Colorbar (single)
        cb = fig.colorbar(cf, axgr.cbar_axes[0], orientation='horizontal', drawedges=True)
        cb.set_label('mm day$^{-1}$')
    
    # Save figure
    plt.savefig(filepath, dpi=300, bbox_inches='tight')

    # Show
    plt.show()